In [ ]:
!pip install vaderSentiment

     |████████████████████████████████| 125 kB 4.1 MB/s 


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
# Create lists to store scraped news urls, headlines and text
url_list = []
date_time = []
news_text = []
headlines = [] 

In [ ]:
for i in range(1,3): #parameters of range function correspond to page numbers in the website with news listings
    #get the list of unique urls in the page
  url = 'https://oilprice.com/Energy/Crude-Oil/Page-{}.html'.format(1)
  request = requests.get(url)
  soup = BeautifulSoup(request.text, "html.parser")
  for links in soup.find_all('div', {'class': 'categoryArticle'}):
      for info in links.find_all('a'):
        if info.get('href') not in url_list:
          url_list.append(info.get('href'))

In [ ]:
for www in url_list:
    #access each url
    headlines.append(www.split("/")[-1].replace('-',' '))
    request = requests.get(www)
    soup = BeautifulSoup(request.text, "html.parser")
    #store date and time of publication of the article
    for dates in soup.find_all('span', {'class': 'article_byline'}):
        date_time.append(dates.text.split('-')[-1])
    
    #store the text of the news
    temp = []
    for news in soup.find_all('p'):
            temp.append(news.text)
    
    #identify the last line of the news article
    for last_sentence in reversed(temp):
        if last_sentence.split(" ")[0]=="By" and last_sentence.split(" ")[-1]=="Oilprice.com":
            break
        elif last_sentence.split(" ")[0]=="By":
            break
    
    #prune non news related text from the scraped data to create the news text
    joined_text = ' '.join(temp[temp.index("More Info")+1:temp.index(last_sentence)])
    news_text.append(joined_text)

In [ ]:
# save news text along with the news headline in a dataframe      
news_df = pd.DataFrame({ 'Date' : date_time,
                         'Headline': headlines,
                         'News': news_text,
                       })

In [ ]:
# use VADER to perform sentiment analysis on stored news articles
analyser = SentimentIntensityAnalyzer()

In [ ]:
def comp_score(text):
   return analyser.polarity_scores(text)["compound"]  

In [ ]:
news_df["sentiment"] = news_df["News"].apply(comp_score)

In [ ]:
news_df

,Date,Headline,News,sentiment
0,"Jan 14, 2022, 12:13 PM CST",US Rig Count Jumps As Crude Rally Continues.html,The number of active drilling rigs in the Unit...,0.8238
1,"Jan 14, 2022, 11:00 AM CST",China Set To Release Crude From Strategic Rese...,China has agreed with the United States to rel...,-0.2732
2,"Jan 14, 2022, 9:00 AM CST",Chinas Crude Imports Fell For The First Time I...,Crude oil imports into China fell by 5.4 perce...,-0.9677
3,"Jan 13, 2022, 5:00 PM CST",US Proved Oil Reserves Slip 19 In 2020.html,Proved oil reserves in the United States slipp...,-0.9434
4,"Jan 13, 2022, 7:00 PM CST",80 Oil Is Too Enticing For US Drillers To Igno...,Despite multiple signals from the U.S. oil ind...,0.9880
5,"Jan 13, 2022, 1:00 PM CST",Libyas Once Great Oil Industry Is Facing A Ple...,There are still high hopes for Libya’s oil ind...,0.6611
6,"Jan 12, 2022, 1:00 PM CST",Chinese Oil Major CNOOC Targets Record High Pr...,China National Offshore Oil Corporation (CNOOC...,0.9673
7,"Jan 12, 2022, 11:00 AM CST",A Watershed Moment That Could Send Oil Prices ...,"Crude oil prices could reach $100 per barrel, ...",0.7759
8,"Jan 12, 2022, 9:38 AM CST",Oil Prices Inch Higher Despite Large Gasoline ...,Crude prices dipped slightly before jumping ag...,-0.4588
9,"Jan 11, 2022, 9:00 AM CST",China Spends Billions To Alleviate Solar Indus...,A global shortage of polysilicon caused by pan...,-0.9505


In [ ]:
from google.colab import files
news_df.to_csv('Sentiment_Analysis.csv')
files.download('Sentiment_Analysis.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>